## Igor - a simple chatbot for wisdom and wellbeing.  
Run all cells, write your thoughts, sip from the cup of the wisdom of the ages.

    Key: clr - clears your input
         otr - records your questions, answer, and rating in a file for your records.
         ftr - stops the recording for complete privacy.

(with acknowledgements to https://medium.com/analytics-vidhya/building-a-simple-chatbot-in-python-using-nltk-7c8c8215ac6e)

In [1]:
##version 003 includes feedback collection to create database for supervised learning of a NN.

import nltk
import numpy as np
import random
import string # to process standard python strings
import re

In [2]:
f=open('cb_compiled.txt','r',errors = 'ignore', encoding = "utf-8")
raw=f.read()
raw=raw.lower()# converts to lowercase
#nltk.download('punkt') # first-time use only
#nltk.download('wordnet') # first-time use only
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words


record_flag=True
#sent_tokens[:2]
#word_tokens[:2]

lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)
        
        
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
f.close()

def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        #randomly pick top 10 answers
        i=-2
        x=0
        robo_responses = []
        track_idxx = [] #to track the array positions of possible answers
        while flat[i]!=0 and x<100:
            idxx=vals.argsort()[0][i]
            
            #print(x, '.  ', sent_tokens[idxx])
            robo_responses.append(sent_tokens[idxx])
            track_idxx.append(idxx)
            i=i-1
            x=x+1
        s = random.randint(0,x-1)
        robo_response = robo_response + robo_responses[s] #robo_response+sent_tokens[idx]
        if record_flag==True: td.write(user_response.strip()+'\t'+str(track_idxx[s])+'\t')
        #print(flat[-2-s]) #checking to see typical flat value.  no insight gained.
        #print(sent_tokens[track_idxx[s]]) #Confirming that idxx is the array position of the reponse in sent_tokens
        return robo_response

In [ ]:
from IPython.display import clear_output

td=open('cb_training_data.txt','a',errors = 'ignore', encoding = "utf-8") #training data for supervised NN


flag=True
print("IGOR: My name is Igor. I will give wisdom, hopefully. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("IGOR: You are welcome..")
        elif(user_response=='off the record' or user_response=='ftr' ):
            record_flag=False
            print("IGOR: You are now OFF the record..")
        elif(user_response=='on the record' or user_response=='ntr' ):
            record_flag=True
            print("IGOR: You are now ON the record..")
        elif(user_response=='clr'):
            clear_output()
        else:
            if(greeting(user_response)!=None):
                print("IGOR: "+greeting(user_response))
            else:
                print("IGOR: ",end="")
                temp_r = response(user_response)
                temp_r = re.sub('-dd-', '.', temp_r)
                temp_r = re.sub('-qq-', '?', temp_r)
                temp_r = re.sub('-xx-', '!', temp_r)
                temp_r = re.sub('â€™', '\'', temp_r)
                
                print(temp_r)
                print('----')
                sent_tokens.remove(user_response)
                if temp_r != "I am sorry! I don't understand you" and record_flag==True:
                    print("good response? (1=worst, 9=best)")
                    feedback_rating = input()
                    td.write(feedback_rating+'\n')
                    print("--------")
    else:
        flag=False
        td.close()
        print("IGOR: Bye! take care..")

i'm afraid that no one will appreciate this little fella
IGOR: be not afraid of going slowly, be afraid only of standing still. chinese proverb .
----
good response? (1=worst, 9=best)
9
--------
ftr
IGOR: You are now OFF the record..
